### 랭체인은 구글 검색, 빙 검색, 덕덕고 검색, 타빌리 검색 등 다양한 도구를 제공하여 인터넷 검색 기능을 추가할 수 있다.

이번 실습에서는 무료로 사용할 수 있는 덕덕고 검색 엔진을 사용하겠다.

In [7]:
import os
from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env")
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [28]:
# 최신 정보를 알려주지 못하는 GPT
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", api_key=OPENAI_API_KEY)
model.invoke("로제의 신곡 APT에 대한 반응은?")

AIMessage(content='죄송하지만, 로제가 발표한 "APT"라는 신곡에 대한 특별한 정보를 가지고 있지 않습니다. 신곡에 대한 반응을 알고 싶다면 소셜 미디어, 음악 리뷰 사이트나 팬 커뮤니티를 통해 최신 반응과 리뷰를 확인해보는 것이 좋을 것 같습니다. 도움이 되길 바랍니다!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 70, 'prompt_tokens': 20, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4poLGKH6aJOT830mXfwVsfezKit7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--58355061-0a0b-4466-ac60-e1c0e4e11848-0', usage_metadata={'input_tokens': 20, 'output_tokens': 70, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### 덕덕고 검색 설치

In [10]:
!pip install duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 70.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [duckduckgo-search]


In [20]:
!pip install ddgs

In [29]:
# 덕덕고 검색 테스트
from langchain_community.tools import DuckDuckGoSearchResults 

search = DuckDuckGoSearchResults(results_separator=';\n')
docs = search.invoke("로제의 신곡 APT에 대한 반응은?")

print(docs)

/home/haiqv/conda/envs/agent-llm/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


snippet: 哔哩哔哩（bilibili.com)是国内知名的视频弹幕网站，这里有及时的动漫新番，活跃的ACG氛围，有创意的Up主。 大家可以在这里找到许多欢乐。, title: 哔哩哔哩 (゜-゜)つロ 干杯~-bilibili, link: https://www.bilibili.com/;
snippet: 哔哩哔哩（bilibili.com)是国内知名的视频弹幕网站，这里有及时的动漫新番，活跃的ACG氛围，有创意的Up主。 大家可以在这里找到许多欢乐。, title: 哔哩哔哩 | 网页版-国内知名的视频弹幕网站, link: https://bilibili.ai-tab.cn/;
snippet: Category - BiliBili, Southeast Asia's leading anime, comics, and games (ACG) community where people can create, watch and share engaging videos., title: Category - BiliBili, link: https://www.bilibili.tv/zh/category;
snippet: bilibili是国内知名的视频弹幕网站，这里有最及时的动漫新番，最棒的ACG氛围，最有创意的Up主。 大家可以在这里找到许多欢乐。, title: 哔哩哔哩, link: https://www.bilibili.com/v/popular/rank/all/


#### 질의 확장 구현하기

In [30]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자의 질문에 대해 아래 context에 기반하여 답변하라.:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = question_answering_prompt | model

In [31]:
# 메시지 저장하고 답변 생성 후 출력하기
from langchain.memory import ChatMessageHistory

# 채팅 메시지를 저장할 메모리 객체 생성
chat_history = ChatMessageHistory() 
# 사용자 질문을 메모리에 저장
chat_history.add_user_message("로제의 신곡 APT에 대한 반응은?") 

# 문서 검색하고 답변 생성
answer = document_chain.invoke(
    {
        "messages": chat_history.messages,
        "context": docs,
    }
)

# 생성된 답변을 메모리에 저장
chat_history.add_ai_message(answer) 

print(answer)

content='죄송하지만, 로제의 신곡 "APT"에 대한 특정 반응은 확인할 수 없습니다. 그러나 빌리빌리는 애니메이션, 만화, 게임과 관련된 다양한 영상을 공유하고, 토론할 수 있는 플랫폼이므로, 이러한 플랫폼에서 팬들의 다양한 의견과 반응을 찾아보시는 것도 좋을 것 같습니다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 361, 'total_tokens': 437, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_46bff0e0c8', 'id': 'chatcmpl-C4poe6qQM8n6aoOpX63oBofSsAnmc', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--f943d0bc-4685-4be3-be96-8a23ce415358-0' usage_metadata={'input_tokens': 361, 'output_tokens': 76, 'total_tokens': 437, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


#### 검색 기능에 옵션 설정하기

In [32]:
# 최신 뉴스 기사만 검색하기
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchResults 

# 한국 지역("kr-kr")을 기준, 최근 일주일("w") 내의 검색 결과를 가져오도록 초기화
wrapper = DuckDuckGoSearchAPIWrapper(region="kr-kr", time="w")

# 뉴스 소스 위주로 검색하도록 지정
# 검색 기능을 위한 DuckDuckGoSearchResults 초기화
search = DuckDuckGoSearchResults(
    api_wrapper=wrapper,      # 앞에서 정의한 API wrapper를 사용
    source="news",            # 뉴스 소스에서만 검색하도록 지정
    results_separator=';\n'   # 결과 항목 사이에 구분자 사용 (세미콜론과 줄바꿈)
)

# 검색
docs = search.invoke("로제의 신곡 APT에 대한 반응은?")

# 검색 결과 출력
print(docs)

/home/haiqv/conda/envs/agent-llm/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


snippet: 4 days ago · 신개념 기초영문법 훈민영음 간지본 2편 로제의 ‘APT.’ 와 윤수일의 ‘아파트’를 비교해보면 문화적 배경에서 비롯한 언어 표현 방식의 차이를 ..., title: 훈민영문법 1분 영문법: 2-1 로제의 "APT." 영어가사의 직설적 표현 ..., link: /videos/riverview/relatedvideo?q=로제의+신곡+APT에+대한+반응은?&filters=ex1:"ez2"&ru=/search?q=%EB%A1%9C%EC%A0%9C%EC%9D%98+%EC%8B%A0%EA%B3%A1+APT%EC%97%90+%EB%8C%80%ED%95%9C+%EB%B0%98%EC%9D%91%EC%9D%80%3F&filters=ex1%3A%22ez2%22&mmscn=vwrc&mid=8F5A57AF4D10983B7A078F5A57AF4D10983B7A07&FORM=WRVORC&ntb=1&msockid=251b035c79e511f0b63f90b59eb7b955;
snippet: 3 days ago · APT (Adversarial PhoneTic Prompting)는 가사의 핵심 구절을 발음과 리듬이 비슷한 다른 단어로 바꾸는 방식입니다. 래퍼 에미넴 (Eminem)의 유명한 랩  가사에 …, title: AI, 블랙핑크와 에미넴 표절 의혹에 휩싸이다, link: https://newneek.co/@mmmya_ai/article/34759;
snippet: 5 days ago · 로제의 히트곡 ‘APT.’ 는 올해의 비디오, 올해의 노래, 최우수 팝 등 7개 부문 후보에 오르며 두각을 나타냈습니다.또한, 데뷔 앨범 “rosie”의 수록곡 ‘ Toxic Till the End ‘는 최우수 K-pop …, title: 로제의 8개 VMA 후보 지명이 그래미상 수상으로 이어질까?, link: https://amkstation.com/ko/로제의-8개-vma-후보-지명이-그래미상-수상으로-이어질까/;
snippet: 4 days ago · 이 밖

In [33]:
# 특정 웹 사이트에서 검색하기
docs = search.invoke("site:ytn.co.kr 갤럭시 폴드7 에 대한 반응은?")
docs

/home/haiqv/conda/envs/agent-llm/lib/python3.11/site-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


'snippet: 블루 쉐도우 색상의 갤럭시 Z 폴드7을 펼친 상태에서 메인 화면을 클로즈업한 모습입니다. 비스듬히 떠 있는 듯한 디자인으로 휴대폰의 매우 얇은 모서리를 보여줍니다., title: 스마트폰 | Samsung 대한민국, link: https://www.samsung.com/sec/smartphones/?msockid=05857eefc74961fe2dcf68a6c63d6053;\nsnippet: 2009년 4월 삼성 갤럭시 (GT-I7500)를 출시하면서 삼성 갤럭시 시리즈가 시작되었고, 이후 갤럭시 스피카 (GT-I5700), 갤럭시 빔 (2010) (GT-I8520) 등의 모델을 출시하면서 갤럭시 시리즈 …, title: 삼성 갤럭시 - 나무위키, link: https://namu.wiki/w/삼성+갤럭시;\nsnippet: Jun 9, 2025 · 2025년 갤럭시 S25 울트라부터 가성비 A시리즈까지! 최신 갤럭시 핸드폰 종류별 가격 비교, 할인 정보, 사용자별 맞춤 추천까지 한번에 확인하세요., title: 2025년 최신 갤럭시 핸드폰 종류, 가격 비교 및 추천 가이드, link: https://www.ajd.co.kr/contents/basic-tip/detail/2025년_최신_갤럭시_핸드폰_종류,_가격_비교_및_추천_가이드-63157;\nsnippet: Jul 17, 2025 · 추가로 삼성 갤럭시 스마트폰은 기능적인 사용에 맞는 폴더블 디스플레이 스마트폰 및 S 펜 활용 제품 및 고급화로 다른 스마트폰과 차별점을 두고 있는 브랜드다.삼성전자의 …, title: 삼성 갤럭시 스마트폰 종류 정리 [ Z 폴드 플립 7 / S 25 엣지 ..., link: https://techwave.tistory.com/entry/Samsung-Galaxy-Smartphone-Types'

In [34]:
# 링크 확인하기

# 검색 결과의 링크들을 저장할 빈 리스트 초기화
links = []

# 검색 결과를 세미콜론과 줄바꿈 기준으로 분리하고, 각 결과 항목에서 링크를 추출
for doc in docs.split(";\n"):
    print(doc)  # 각 검색 결과 항목을 출력하여 확인
    link = doc.split("link:")[1].strip()  # 각 항목에서 'link:' 이후의 URL 부분만 추출
    links.append(link)  # 추출한 링크를 리스트에 추가

# 모든 링크를 출력
print(links)

snippet: 블루 쉐도우 색상의 갤럭시 Z 폴드7을 펼친 상태에서 메인 화면을 클로즈업한 모습입니다. 비스듬히 떠 있는 듯한 디자인으로 휴대폰의 매우 얇은 모서리를 보여줍니다., title: 스마트폰 | Samsung 대한민국, link: https://www.samsung.com/sec/smartphones/?msockid=05857eefc74961fe2dcf68a6c63d6053
snippet: 2009년 4월 삼성 갤럭시 (GT-I7500)를 출시하면서 삼성 갤럭시 시리즈가 시작되었고, 이후 갤럭시 스피카 (GT-I5700), 갤럭시 빔 (2010) (GT-I8520) 등의 모델을 출시하면서 갤럭시 시리즈 …, title: 삼성 갤럭시 - 나무위키, link: https://namu.wiki/w/삼성+갤럭시
snippet: Jun 9, 2025 · 2025년 갤럭시 S25 울트라부터 가성비 A시리즈까지! 최신 갤럭시 핸드폰 종류별 가격 비교, 할인 정보, 사용자별 맞춤 추천까지 한번에 확인하세요., title: 2025년 최신 갤럭시 핸드폰 종류, 가격 비교 및 추천 가이드, link: https://www.ajd.co.kr/contents/basic-tip/detail/2025년_최신_갤럭시_핸드폰_종류,_가격_비교_및_추천_가이드-63157
snippet: Jul 17, 2025 · 추가로 삼성 갤럭시 스마트폰은 기능적인 사용에 맞는 폴더블 디스플레이 스마트폰 및 S 펜 활용 제품 및 고급화로 다른 스마트폰과 차별점을 두고 있는 브랜드다.삼성전자의 …, title: 삼성 갤럭시 스마트폰 종류 정리 [ Z 폴드 플립 7 / S 25 엣지 ..., link: https://techwave.tistory.com/entry/Samsung-Galaxy-Smartphone-Types
['https://www.samsung.com/sec/smartphones/?msockid=05857eefc74961fe2dcf68a6c63d6053', 'https://namu

In [35]:
# 웹 페이지의 내용을 비동기로 읽어 오기
# Langchain의 WebBaseLoader를 사용하여 웹 페이지의 내용을 불러옵니다.
from langchain_community.document_loaders import WebBaseLoader

# WebBaseLoader 객체를 생성. 'links'는 웹 페이지의 URL 목록을 담고 있는 변수
# bs_get_text_kwargs는 BeautifulSoup의 get_text() 메소드에 전달될 추가 인자
loader = WebBaseLoader(
    web_paths=links,  # 웹 페이지의 링크 목록을 지정
    bs_get_text_kwargs={
        "strip": True  # 웹 페이지에서 텍스트를 가져올 때 앞뒤의 공백을 제거
    },
)

# 비동기로 웹 페이지의 내용을 로드하고, 각 문서를 page_contents 리스트에 추가
page_contents = []  # 각 웹 페이지의 내용을 저장할 리스트입니다.
async for doc in loader.alazy_load():
    page_contents.append(doc)  # 불러온 문서를 page_contents 리스트에 추가

# page_contents에 있는 각 웹 페이지의 내용을 출력
for content in page_contents:
    print(content)  # 웹 페이지의 내용을 출력
    print('--------------')  # 페이지 구분을 위해 구분선을 출력

USER_AGENT environment variable not set, consider setting it to identify your requests.
Fetching pages: 100%|##########| 4/4 [00:00<00:00,  8.69it/s]


page_content='스마트폰 | Samsung 대한민국본문 바로가기일주일 그만보기닫기갤럭시 AI 폰구매 고객님께 드리는스마트태그2를 지금 신청하세요!(대상 모델 : S24 시리즈, S23 FE, Z Fold5 | Z Flip5, S23 시리즈)신청하기갤럭시 캠퍼스 회원이시네요! 리뉴얼 이벤트와오리지널 콘텐츠, 더 풍성해진 커뮤니티까지!새로워진 갤럭시 캠퍼스를 만나보세요갤럭시 캠퍼스로 이동하기갤럭시 Z 폴드7기획전기획전모바일모바일TV/영상∙음향TV/영상∙음향주방가전주방가전리빙가전리빙가전PC/주변기기PC/주변기기웨어러블웨어러블소모품/액세서리소모품/액세서리AI 구독클럽AI 구독클럽혜택/추천 더 알아보기삼성닷컴에서 더 많은 혜택을삼성닷컴 앱 다운로드스마트싱스삼성케어플러스 모바일/PC삼성케어플러스 가전/TVAI for All갤럭시 캠퍼스갤럭시와 함께하는 AI 클래스고객지원고객지원비즈니스사업자몰삼성스토어삼성스토어지속가능경영주문/배송조회멤버십나의 제품 관리쿠폰존삼성닷컴 회원 혜택참여형 이벤트갤럭시 Z 폴드7뒤로가기홈으로 가기검색버튼장바구니 버튼유저 메뉴버튼메뉴전체삭제검색 하기검색창 닫기검색결과가 없습니다.다른 검색어 or 인기 검색어를 확인해보세요!최근 본 제품최근 본 제품 닫기검색 결과0개전체삭제※ 최근 본 제품은 30일간 보관됩니다.모바일이전갤럭시 스마트폰갤럭시 탭갤럭시 북갤럭시 워치갤럭시 버즈갤럭시 링갤럭시 액세서리다음갤럭시 Z대화면과 폴더블로확장되는 AI 경험갤럭시 S강력한 칩셋과 카메라,진화된 맞춤형 AI 경험갤럭시 A멀티 카메라와고성능 배터리비교하기나에게 딱 맞는갤럭시를 찾아보세요액세서리함께 쓰면 더 좋은스마트폰 액세서리전체 스마트폰스마트폰모두 보기갤럭시 Z 폴드7, 울트라를 펼치다더 알아보기구매하기블루 쉐도우 색상의 갤럭시 Z 폴드7을 펼친 상태에서 메인 화면을 클로즈업한 모습입니다. 비스듬히 떠 있는 듯한 디자인으로 휴대폰의 매우 얇은 모서리를 보여줍니다. Galaxy AI 로고가 함께 표시되어 있습니다.진정한 AI 폰,삼성닷컴 | 삼성 강남 전용 컬러